In [1]:
import sys
sys.path.insert(0, '../../../network')

In [2]:
import logging
import argparse
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

In [3]:
As = [np.array([5, 5, i, 1]) for i in np.arange(0, 3, 0.5)]
for A in As:
    A_cc, A_bb, A_cb, A_bc = round(A[0], 1), round(A[1], 1), round(A[2], 1), round(A[3], 1)
    
    phi = ErrorFunction(mu=0.22, sigma=0.1).phi
    ctx = Population(N=1000, tau=1e-2, phi=phi, name='ctx')
    bg = Population(N=1000, tau=1e-2, phi=phi, name='bg')
    plasticity = ThresholdPlasticityRule(x_f=0.5, q_f=0.8) 

    S, P = 1, 3
    sequences_ctx = [GaussianSequence(P,ctx.size, seed=5) for i in range(S)]
    patterns_ctx = np.stack([s.inputs for s in sequences_ctx])
    sequences_bg = [GaussianSequence(P,ctx.size, seed=27) for i in range(S)]
    patterns_bg = np.stack([s.inputs for s in sequences_bg])
    
    J_cc = SparseConnectivity(source=ctx, target=ctx, p=0.05)
    synapse_cc = LinearSynapse(J_cc.K, A=A_cc)
    J_cc.store_attractors(patterns_ctx[0], patterns_ctx[0], synapse_cc.h_EE, 
                          plasticity.f, plasticity.g)
    J_bb = SparseConnectivity(source=bg, target=bg, p=0.05)
    synapse_bb = LinearSynapse(J_bb.K, A=A_bb)
    J_bb.store_attractors(patterns_bg[0], patterns_bg[0], synapse_bb.h_EE, 
                          plasticity.f, plasticity.g)
    J_cb  = SparseConnectivity(source=bg, target=ctx, p=0.05)
    synapse_cb = LinearSynapse(J_cb.K, A=A_cb)
    J_cb.store_attractors(patterns_bg[0], patterns_ctx[0], synapse_cb.h_EE, 
                          plasticity.f, plasticity.g)
    J_bc = SparseConnectivity(source=ctx, target=bg, p=0.05)
    synapse_bc = LinearSynapse(J_cc.K, A=0)
    J_bc.store_sequences(patterns_ctx, patterns_bg, synapse_bc.h_EE, plasticity.f, plasticity.g)
    J_bc.update_sequences(patterns_ctx[0][0], patterns_bg[0][1],
                       A_bc, lamb=1,f=plasticity.f, g=plasticity.g)
    J_bc.update_sequences(patterns_ctx[0][1], patterns_bg[0][2],
                       A_bc, lamb=1,f=plasticity.f, g=plasticity.g)
    J_bc.update_sequences(patterns_ctx[0][2], patterns_bg[0][0],
                       A_bc, lamb=1,f=plasticity.f, g=plasticity.g)
    net_ctx = RateNetwork(ctx, c_EE=J_cc, c_IE=J_bc, formulation=4)
    net_bg = RateNetwork(bg, c_II=J_bb, c_EI=J_cb, formulation=4)
    
    init_input_ctx = phi(patterns_ctx[0][0])
    init_input_bg = phi(patterns_bg[0][0])
    T=2
    net_ctx.simulate_euler2(net_bg, T, init_input_ctx, init_input_bg)
    overlaps_ctx = sequences_ctx[0].overlaps(net_ctx, ctx, phi=phi)
    overlaps_bg = sequences_bg[0].overlaps(net_bg, bg, phi=phi)
    corr_ctx = sequences_ctx[0].overlaps(net_ctx, ctx, phi=phi, correlation=True)
    corr_bg = sequences_ctx[0].overlaps(net_bg, bg, phi=phi, correlation=True)
    np.savez('./output/'+str(A_cc)+'-'+str(A_bb)+'-'+str(A_cb)+'-'+str(A_bc)+'.npz', 
         overlaps_ctx=overlaps_ctx, overlaps_bg=overlaps_bg, corr_ctx=corr_ctx, corr_bg=corr_bg)


INFO:connectivity:Building connections from ctx to ctx
INFO:connectivity:Storing attractors
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 33742.31it/s]
INFO:connectivity:Building connections from bg to bg
INFO:connectivity:Storing attractors
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 30396.15it/s]
INFO:connectivity:Building connections from bg to ctx
INFO:connectivity:Storing attractors
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 32362.71it/s]
INFO:connectivity:Building connections from ctx to bg
INFO:connectivity:Storing sequences
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 35.64it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix
/home/jp464/Desktop/SM/hebbian_sequence_learning/figures/article/1/../../../network/transfer_functions.py:125: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  return 1 

INFO:connectivity:Storing attractors
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 38881.15it/s]
INFO:connectivity:Building connections from bg to bg
INFO:connectivity:Storing attractors
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 38611.98it/s]
INFO:connectivity:Building connections from bg to ctx
INFO:connectivity:Storing attractors
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 38349.32it/s]
INFO:connectivity:Building connections from ctx to bg
INFO:connectivity:Storing sequences
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 35.49it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix
INFO:network:Integrating network dynamics
100%|██████████████████████████████████████| 1999/1999 [00:02<00:00, 955.77it/s]
INFO:sequences:Computing overlaps
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1553.83it/s]
INFO:sequences:Computing overlaps
1